## Second iteration

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from autograd import value_and_grad
import matplotlib.pyplot as plt



df = pd.read_csv('dynamic_pricing.csv')

# Preprocessing
categorical_features = ['Location_Category', 'Customer_Loyalty_Status', 'Time_of_Booking', 'Vehicle_Type']
numerical_features = ['Number_of_Riders', 'Number_of_Drivers', 'Number_of_Past_Rides', 'Average_Ratings', 'Expected_Ride_Duration']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Apply the transformations to the dataframe
X = df.drop(columns=['Historical_Cost_of_Ride'])
y = df['Historical_Cost_of_Ride'].values

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and transform the training data, transform the test data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Add bias term (intercept) to the features
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# Reshape for the model
X_train = X_train.T[np.newaxis, :]
X_test = X_test.T[np.newaxis, :]
y_train = y_train[np.newaxis, :]
y_test = y_test[np.newaxis, :]

# Model and cost function
def model(w, x):
    return np.dot(x[0].T, w)

def least_squares(w, x, y):
    cost = np.sum((model(w, x) - y) ** 2)
    return cost

def gradient_descent(g, alpha, max_its, w, x, y):
    gradient = value_and_grad(g)
    weight_history = []
    cost_history = []
    for k in range(1, max_its + 1):
        cost_eval, grad_eval = gradient(w, x, y)
        weight_history.append(w)
        cost_history.append(cost_eval)
        w = w - alpha * grad_eval
    weight_history.append(w)
    cost_history.append(g(w, x, y))
    cost_history = np.asarray(cost_history) / x.shape[2]
    return weight_history, cost_history

# Training the model
rate = 1e-5
max_it = 500
initial_weights = np.random.rand(X_train.shape[1])
wh, ch = gradient_descent(least_squares, rate, max_it, initial_weights, X_train, y_train)

# Finding the best weights
index = np.argmin(ch)
w_star = wh[index]

# Evaluating the model
cost_train = ch[index]
cost_test = least_squares(w_star, X_test, y_test) / X_test.shape[2]

print('Optimized Weights: ', np.round(w_star, 1))
print('Optimized Cost: {:.2f}'.format(cost_train))
print('Test Cost: {:.2f}'.format(cost_test))

# Calculate APP for static and dynamic pricing
def demand_curve(axis, value):
    demand = []
    sample_size = len(value)
    for price in axis:
        demand.append(sum(1 if x >= price else 0 for x in value) / sample_size)
    return demand

def APP_s(axis, value, c):
    demand = demand_curve(axis, value)
    app_s = []
    for i in range(len(axis)):
        app_s.append(max(axis[i] - c, 0) * demand[i])
    return app_s

def APP_d(w, x, y, c, d):
    est = d * model(w, x)
    ind = list(range(len(est)))
    rev = sum(max(est[i], c) if y[0][i] >= max(est[i], c) else 0 for i in ind)
    return rev / x.shape[2]

c = 0  # Assuming cost is 0 for simplicity
axis_2 = np.linspace(0, int(np.round(np.max(y_train), 0) + 1), int(2 * (np.round(np.max(y_train), 0) + 1)))
app_s_train = APP_s(axis_2, y_train[0], c)
index = np.argmax(app_s_train)
app_s_test = APP_s(axis_2, y_test[0], c)
app_s_star = app_s_test[index]

print('Optimized Static APP: {:.2f}'.format(app_s_star))
print('Optimized Price:', axis_2[index])

app_d_1 = APP_d(w_star, X_test, y_test, c, 1)
print('Dynamic APP: {:.2f}'.format(app_d_1))

axis_d = np.linspace(0.7, 1, 16)
app_d = [APP_d(w_star, X_test, y_test, c, d) for d in axis_d]

d_star = axis_d[np.argmax(app_d)]
app_d_star = APP_d(w_star, X_train, y_train, c, d_star)

print('Optimized Discount: {:.2f}'.format(d_star))
print('Test Dynamic APP: {:.2f}'.format(app_d_star))

print('Dynamic APPU (Test-based): {:.2f}'.format(app_d_star))
print('Static APPU (Test-based): {:.2f}'.format(app_s_star))
print('Percentage in Profit Increase: {:.2%}'.format((app_d_star - app_s_star) / app_s_star))

plt.figure()
plt.plot(range(max_it + 1), ch)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Controller Training')
plt.show()

plt.figure()
plt.plot(axis_2, app_s_train)
plt.xlabel('Price of Good')
plt.ylabel('APP')
plt.title('Static Price Profit Maximization')
plt.show()

plt.figure()
plt.plot(axis_d, app_d)
plt.xlabel('Discount')
plt.ylabel('APP')
plt.title('Dynamic Price Profit Maximization')
plt.show()


KeyboardInterrupt: 

## First iteration

In [1]:
import autograd.numpy as np
from autograd import value_and_grad
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
# Assuming your data is in a pandas DataFrame
df = pd.read_csv('dynamic_pricing.csv')

# Preprocessing
categorical_features = ['Location_Category', 'Customer_Loyalty_Status', 'Vehicle_Type']
numerical_features = ['Number_of_Riders', 'Number_of_Drivers', 'Number_of_Past_Rides', 'Average_Ratings', 'Time_of_Booking', 'Expected_Ride_Duration']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [4]:



# Apply the transformations to the dataframe
X = df.drop(columns=['Historical_Cost_of_Ride'])
y = df['Historical_Cost_of_Ride'].values

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and transform the training data, transform the test data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Add bias term (intercept) to the features
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# Reshape for the model
X_train = X_train.T[np.newaxis, :]
X_test = X_test.T[np.newaxis, :]
y_train = y_train[np.newaxis, :]
y_test = y_test[np.newaxis, :]


ValueError: could not convert string to float: 'Night'

In [ ]:

# Model and cost function
def model(w, x):
    return np.dot(x[0].T, w)

def least_squares(w, x, y):
    cost = np.sum((model(w, x) - y) ** 2)
    return cost

def gradient_descent(g, alpha, max_its, w, x, y):
    gradient = value_and_grad(g)
    weight_history = []
    cost_history = []
    for k in range(1, max_its + 1):
        cost_eval, grad_eval = gradient(w, x, y)
        weight_history.append(w)
        cost_history.append(cost_eval)
        w = w - alpha * grad_eval
    weight_history.append(w)
    cost_history.append(g(w, x, y))
    cost_history = np.asarray(cost_history) / x.shape[2]
    return weight_history, cost_history

# Training the model
rate = 1e-5
max_it = 500
initial_weights = np.random.rand(X_train.shape[1])
wh, ch = gradient_descent(least_squares, rate, max_it, initial_weights, X_train, y_train)

# Finding the best weights
index = np.argmin(ch)
w_star = wh[index]

# Evaluating the model
cost_train = ch[index]
cost_test = least_squares(w_star, X_test, y_test) / X_test.shape[2]

print('Optimized Weights: ', np.round(w_star, 1))
print('Optimized Cost: {:.2f}'.format(cost_train))
print('Test Cost: {:.2f}'.format(cost_test))

# Calculate APP for static and dynamic pricing
def APP_s(axis, value, c):
    demand = demand_curve(axis, value)
    app_s = []
    for i in range(len(axis)):
        app_s.append(max(axis[i] - c, 0) * demand[i])
    return app_s

def demand_curve(axis, value):
    demand = []
    sample_size = len(value)
    for price in axis:
        demand.append(sum(1 if x >= price else 0 for x in value) / sample_size)
    return demand

def APP_d(w, x, y, c, d):
    est = d * model(w, x)
    ind = list(range(len(est)))
    rev = sum(max(est[i], c) if y[0][i] >= max(est[i], c) else 0 for i in ind)
    return rev / x.shape[2]

c = 0  # Assuming cost is 0 for simplicity
axis_2 = np.linspace(0, int(np.round(np.max(y_train), 0) + 1), int(2 * (np.round(np.max(y_train), 0) + 1)))
app_s_train = APP_s(axis_2, y_train[0], c)
index = np.argmax(app_s_train)
app_s_test = APP_s(axis_2, y_test[0], c)
app_s_star = app_s_test[index]

print('Optimized Static APP: {:.2f}'.format(app_s_star))
print('Optimized Price:', axis_2[index])

app_d_1 = APP_d(w_star, X_test, y_test, c, 1)
print('Dynamic APP: {:.2f}'.format(app_d_1))

axis_d = np.linspace(0.7, 1, 16)
app_d = [APP_d(w_star, X_test, y_test, c, d) for d in axis_d]

d_star = axis_d[np.argmax(app_d)]
app_d_star = APP_d(w_star, X_train, y_train, c, d_star)

print('Optimized Discount: {:.2f}'.format(d_star))
print('Test Dynamic APP: {:.2f}'.format(app_d_star))

print('Dynamic APPU (Test-based): {:.2f}'.format(app_d_star))
print('Static APPU (Test-based): {:.2f}'.format(app_s_star))
print('Percentage in Profit Increase: {:.2%}'.format((app_d_star - app_s_star) / app_s_star))

plt.figure()
plt.plot(range(max_it + 1), ch)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Controller Training')
plt.show()

plt.figure()
plt.plot(axis_2, app_s_train)
plt.xlabel('Price of Good')
plt.ylabel('APP')
plt.title('Static Price Profit Maximization')
plt.show()

plt.figure()
plt.plot(axis_d, app_d)
plt.xlabel('Discount')
plt.ylabel('APP')
plt.title('Dynamic Price Profit Maximization')
plt.show()
